In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import pandas as pd

from gzreduction.vote_sources.panoptes_exports import export_to_responses
from gzreduction.vote_sources.panoptes_exports import aggregate
from gzreduction.schemas.euclid_schema import euclid_schema

In [3]:

schema = euclid_schema

In [4]:
classification_export_loc = '/home/walml/repos/gz-panoptes-reduction/data/euclid/euclid-development-classifications.csv'
aggregated_loc = '/home/walml/repos/gz-panoptes-reduction/data/euclid/euclid_development_reduced_votes.csv'

In [5]:
# nrows = 10000
nrows = None
clf = pd.read_csv(classification_export_loc, nrows=nrows)
clf

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,558337274,MikeWalmsley,290475,a71cb7b76ff9e1068c18,26524,Euclid development,2.3,2024-04-29 20:48:53 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765768"":{""retired"":null,""Filename"":""resize...",98765768
1,558337318,MikeWalmsley,290475,a71cb7b76ff9e1068c18,26524,Euclid development,2.3,2024-04-29 20:49:03 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765875"":{""retired"":null,""Filename"":""resize...",98765875
2,558337354,MikeWalmsley,290475,a71cb7b76ff9e1068c18,26524,Euclid development,2.3,2024-04-29 20:49:11 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765809"":{""retired"":null,""Filename"":""resize...",98765809
3,558337455,MikeWalmsley,290475,a71cb7b76ff9e1068c18,26524,Euclid development,2.3,2024-04-29 20:49:34 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765789"":{""retired"":null,""Filename"":""resize...",98765789
4,558337573,MikeWalmsley,290475,ff6f67399b8072549a4a,26524,Euclid development,2.3,2024-04-29 20:50:07 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765949"":{""retired"":null,""Filename"":""resize...",98765949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,558342805,MikeWalmsley,290475,28ecb45b81698288898c,26524,Euclid development,2.3,2024-04-29 21:12:13 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765551"":{""retired"":null,""Filename"":""resize...",98765551
96,558342830,MikeWalmsley,290475,28ecb45b81698288898c,26524,Euclid development,2.3,2024-04-29 21:12:18 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765252"":{""retired"":null,""Filename"":""resize...",98765252
97,558342864,MikeWalmsley,290475,28ecb45b81698288898c,26524,Euclid development,2.3,2024-04-29 21:12:27 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765458"":{""retired"":null,""Filename"":""resize...",98765458
98,558342886,MikeWalmsley,290475,28ecb45b81698288898c,26524,Euclid development,2.3,2024-04-29 21:12:33 UTC,NaN,NaN,"{""source"":""api"",""session"":""13a7a7c66fdfdbe7a9a...","[{""task"":""T0"",""task_label"":""Is the galaxy simp...","{""98765243"":{""retired"":null,""Filename"":""resize...",98765243


In [6]:
def get_euclid_subject_id(subject_data):
    loaded = json.loads(subject_data)
    inner_dict = loaded[list(loaded.keys())[0]]
    return inner_dict['Filename']

In [8]:
clf['id_str'] = clf['subject_data'].apply(get_euclid_subject_id)
clf['id_str'] 

0     resized_102022523_10235.jpg
1     resized_102022523_11497.jpg
2     resized_102022523_10778.jpg
3     resized_102022523_10398.jpg
4     resized_102022523_12585.jpg
                 ...             
95     resized_102022523_7580.jpg
96     resized_102022523_4028.jpg
97     resized_102022523_6153.jpg
98     resized_102022523_3998.jpg
99     resized_102022523_9296.jpg
Name: id_str, Length: 100, dtype: object

In [9]:
workflows = [26524]
clf = clf[clf['workflow_id'].isin(workflows)]

In [10]:
df = export_to_responses.explode_annotations(clf, exclude_tasks=['T10'])
df

,task,task_label,value,id_str,user_id,classification_id,created_at,subject_ids,workflow_version
0,T0,"Is the galaxy simply smooth and rounded, with ...",![features_or_disk_new.png](https://panoptes-u...,resized_102022523_10235.jpg,290475,558337274,2024-04-29 20:48:53 UTC,98765768,2.3
0,T2,Could this be a disk viewed edge-on?,![feature_or_disk.png](https://panoptes-upload...,resized_102022523_10235.jpg,290475,558337274,2024-04-29 20:48:53 UTC,98765768,2.3
0,T4,Is there a bar feature through the centre of t...,![bar_none_sidebyside_acwacw_200x100.png](http...,resized_102022523_10235.jpg,290475,558337274,2024-04-29 20:48:53 UTC,98765768,2.3
0,T5,Is there any sign of a spiral arm pattern?,![rounded.png](https://panoptes-uploads.zooniv...,resized_102022523_10235.jpg,290475,558337274,2024-04-29 20:48:53 UTC,98765768,2.3
0,T8,"Is there a central bulge? If so, how large is ...",![bulge_icon_small_bulge.png](https://panoptes...,resized_102022523_10235.jpg,290475,558337274,2024-04-29 20:48:53 UTC,98765768,2.3
...,...,...,...,...,...,...,...,...,...
99,T6,How tightly wound do the spiral arms appear?,![mid_arms.png](https://panoptes-uploads.zooni...,resized_102022523_9296.jpg,290475,558342969,2024-04-29 21:12:53 UTC,98765685,2.3
99,T7,How many spiral arms are there?,![3_arms.png](https://panoptes-uploads.zoonive...,resized_102022523_9296.jpg,290475,558342969,2024-04-29 21:12:53 UTC,98765685,2.3
99,T8,"Is there a central bulge? If so, how large is ...",![bulge_icon_moderate_bulge.png](https://panop...,resized_102022523_9296.jpg,290475,558342969,2024-04-29 21:12:53 UTC,98765685,2.3
99,T13,Are there any obvious bright clumps?,![no.png](https://panoptes-uploads.zooniverse....,resized_102022523_9296.jpg,290475,558342969,2024-04-29 21:12:53 UTC,98765685,2.3


In [11]:
question = schema.get_question_from_raw_name('T0')
# question.get_answer_names()
question.get_raw_answer_names()

['smooth', 'features or disk', 'star, artifact, or bad zoom']

In [12]:
cleaned = export_to_responses.clean_exploded_annotations(df, schema)

In [13]:
aggregated = aggregate.responses_to_reduced_votes(cleaned.reset_index(drop=True), schema)

/home/walml/repos/gz-panoptes-reduction/gzreduction/vote_sources/panoptes_exports/aggregate.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[question.get_fraction_column(answer)] = df[question.get_count_column(answer)] / df[question.total_votes]  # may give nans?
/home/walml/repos/gz-panoptes-reduction/gzreduction/vote_sources/panoptes_exports/aggregate.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[question.get_fraction_column(answer)] = df[question.get_count_column(answer)] / df[question.total_votes]  # may giv

In [14]:
aggregated

question_response,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_problem,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,problem_artifact_fraction,problem_zoom_fraction,artifact_non-star_fraction,artifact_satellite_fraction,artifact_scattered_fraction,artifact_diffraction_fraction,artifact_ray_fraction,artifact_saturation_fraction,artifact_other_fraction,artifact_ghost_fraction
0,resized_102022523_10164.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,resized_102022523_1023.jpg,1,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,resized_102022523_10235.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,resized_102022523_10263.jpg,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,resized_102022523_10354.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,resized_102022523_9407.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,resized_102022523_9516.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,resized_102022523_9780.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,resized_102022523_989.jpg,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
aggregated

question_response,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_problem,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,problem_artifact_fraction,problem_zoom_fraction,artifact_non-star_fraction,artifact_satellite_fraction,artifact_scattered_fraction,artifact_diffraction_fraction,artifact_ray_fraction,artifact_saturation_fraction,artifact_other_fraction,artifact_ghost_fraction
0,resized_102022523_10164.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,resized_102022523_1023.jpg,1,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,resized_102022523_10235.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,resized_102022523_10263.jpg,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,resized_102022523_10354.jpg,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,resized_102022523_9407.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,resized_102022523_9516.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,resized_102022523_9780.jpg,0,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,resized_102022523_989.jpg,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
aggregated.to_csv(aggregated_loc, index=False)